In [4]:
!pip3 install -r requirements.txt
!pip3 install datasets trl peft bitsandbytes wandb accelerate
!pip3 install haystack-lemmatize-node

     |████████████████████████████████| 12.8 MB 5.9 MB/s eta 0:00:01
  Using cached fsspec-2024.6.0-py3-none-any.whl (176 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.5.0
    Uninstalling fsspec-2024.5.0:
      Successfully uninstalled fsspec-2024.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.20.0 requires fsspec[http]<=2024.5.0,>=2023.1.0, but you have fsspec 2024.6.0 which is incompatible.
  Using cached fsspec-2024.5.0-py3-none-any.whl (316 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.0
    Uninstalling fsspec-2024.6.0:
      Successfully uninstalled fsspec-2024.6.0
ERROR: Could not find a version that satisfies the requirement haystack-lemmatize-node
ERROR: No matching distribution found for haystack-lemmatize-node


In [6]:
!pwd

/home/shayan283/Projects/sindri


In [1]:
import torch

from haystack import Pipeline
from haystack.components.preprocessors import TextCleaner, DocumentCleaner, DocumentSplitter
from haystack.components.converters import HTMLToDocument
from huggingface_hub import notebook_login

from haystack.components.builders import PromptBuilder
from haystack.components.preprocessors import TextCleaner
#from haystack.components.generators import HuggingFaceLocalChatGenerator
from transformers import BitsAndBytesConfig

from components import Lemmatizer, Scraper, Text2Doc

In [3]:
notebook_login()

In [49]:


prompt_template = """
    <|begin_of_text|><|start_header_id|>user<|end_header_id|>
	You are a privacy policy analyzer. Your job is to evaluate the given set of documents and list
	any personally identifiable information that is collected by the third party service, which
	includes details about the user themselves (such as their name and address), or information
	about their device(s) (such as their geographic location, device ID, advertising ID, device
 	settings or configurations, network details such as WiFi SSIDs, Interet Protocol Addresses,
  	MAC Addresses) This detail can also include information collected automatically by the service
   provider about user interactions, such as image and audio information, any type of metadata,
   cookies, and tracking across other websites and applications.

   Analyze these documents with this criteria:
   {% for doc in documents %}
        {{ doc.content }}
    {% endfor %}

    <|start_header_id|>assistant<|end_header_id|>
"""

pre_processing_pipeline = Pipeline()
pre_processing_pipeline.add_component("scraper", Scraper())
pre_processing_pipeline.add_component("text_cleaner", TextCleaner(remove_numbers=False, remove_regexps=['[^A-Za-z0-9 !./:-?&%=]'], convert_to_lowercase=True))
pre_processing_pipeline.add_component("lemmatizer", Lemmatizer())
pre_processing_pipeline.add_component("text2doc", Text2Doc())
pre_processing_pipeline.add_component("document_splitter", DocumentSplitter(split_by="sentence", split_length=5, split_overlap=0))
pre_processing_pipeline.connect("scraper", "text_cleaner")
pre_processing_pipeline.connect("text_cleaner", "lemmatizer.text")
pre_processing_pipeline.connect("lemmatizer", "text2doc.text")
pre_processing_pipeline.connect("text2doc", "document_splitter")
#pre_processing_pipeline.add_component("prompt_builder", PromptBuilder(template=prompt_template))

🚅 Components
  - scraper: Scraper
  - text_cleaner: TextCleaner
  - lemmatizer: Lemmatizer
  - text2doc: Text2Doc
  - document_splitter: DocumentSplitter
🛤️ Connections
  - scraper.text -> text_cleaner.texts (List[str])
  - text_cleaner.texts -> lemmatizer.text (List[str])
  - lemmatizer.text -> text2doc.text (List[str])
  - text2doc.document -> document_splitter.documents (List[Document])

In [51]:
result = pre_processing_pipeline.run({"scraper":{"url": "https://policies.google.com/privacy?hl=en-US#products"}})
#print('\n'.join())
documents = [doc.content for doc in result["document_splitter"]["documents"]]